In [ ]:
ABSOLUTE_PATH = os.path.dirname(os.path.abspath(__file__))
MOVES_FOLDER_PATH = os.path.join(ABSOLUTE_PATH, 'games')

MOVES_JSON_FILENAME = '0000.json'

try:
    moves_file_path = os.path.join(MOVES_FOLDER_PATH, MOVES_JSON_FILENAME)
    moves = loadMoves(moves_file_path)
except:
    print("Error loading moves file")

# Load moves from JSON file
def loadMoves(filepath):
    out = []
    with open(filepath, 'rb') as f:
        data = json.load(f)
        for move in data:
            heldBlock = move["heldBlock"] #I|O|T|L|J|S|Z|None|
            activeBlock = move["activeBlock"] #I|O|T|L|J|S|Z
            blockPos = [activeBlock["pos"]["x"], activeBlock["pos"]["y"]]
            blockRot = activeBlock["rot"] #0|1|2|3
            currGrid = move["oldBoard"]
            nextMove = move["move"] #MOVE_LEFT | MOVE_RIGHT | ROT_CCW | ROT_CW | ROT_180 | SOFT_DROP | HARD_DROP
            console.log(heldBlock, activeBlock, blockPos, blockRot, nextMove)
    return out

: 